In [ ]:
import numpy as np
import csv
import pandas as pd
from datetime import datetime
from pandas import read_csv
import missingno as msno
from sklearn.metrics import mean_squared_error, mean_absolute_error 
from math import sqrt
from keras import optimizers
import matplotlib.pyplot as plt, seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import os
import random
import time

In [ ]:
#Load data
df=pd.read_excel("./Dataset/solar.xlsx")

In [ ]:
df.head()

In [ ]:
#Drop all nan values
df=df.dropna()

In [ ]:
df.isna().sum()

In [ ]:
sns.boxplot(x='Production',data=df)

In [ ]:
plt.subplots(figsize=(16, 16))
sns.heatmap(df.corr(), annot=True, square=True)
plt.show()

In [ ]:
1#evaluation metrics
def forecast_accuracy(forecast, actual):
    forecast = np.array(forecast)
    actual = np.array(actual)
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual)) # MAPE
    if (type(mape) != int or type(mape) != float):
        mape='Zero Devision'
    mae = np.mean(np.abs(forecast - actual))    # MAE
    rmse = np.mean((forecast - actual)**2)**.5# RMSE
    if (max(actual)==min(actual)):
        nrmse='Zero Devision'
    else:
        nrmse=rmse/(max(actual)-min(actual))
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    mins = np.amin(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    maxs = np.amax(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    minmax = 1 - np.mean(mins/maxs)             # minmax
    if (type(minmax) != int or type(minmax) != float):
        minmax='Zero Devision'
    return({'mape':mape,'nrmse':nrmse, 'mae': mae, 
            'rmse':rmse, 'corr':corr, 'minmax':minmax})

In [ ]:
def create_X_Y(ts: np.array, lag=1, n_ahead=1, target_index=0) -> tuple:
    """
    A method to create X and Y matrix from a time series array for the training of 
    deep learning models 
    """
    # Extracting the number of features that are passed from the array 
    n_features = ts.shape[1]
    
    # Creating placeholder lists
    X, Y = [], []

    if len(ts) - lag <= 0:
        X.append(ts)
    else:
        for i in range(len(ts) - lag - n_ahead):
            Y.append(ts[(i + lag):(i + lag + n_ahead), target_index])
            X.append(ts[i:(i + lag)])

    X, Y = np.array(X), np.array(Y)

    # Reshaping the X array to an LSTM input shape 
    X = np.reshape(X, (X.shape[0], lag, n_features))

    return X, Y

In [ ]:
# Number of lags to use for models
lag = 24
# Steps in future to forecast
n_ahead = 24
# ratio of observations for training from total series
train_share = 0.8
# training epochs
epochs = 50
# Batch size , which is the number of samples of lags
batch_size = 512
# Learning rate
lr = 0.001
# The features for the modeling 
features_final = ['Production','ensoleillement','t2m','Panel','u10ff']

ts = df[features_final]

In [ ]:
#Scaling data between 0 and 1
scaler = MinMaxScaler()
scaler.fit(ts)
ts_scaled = scaler.transform(ts)

In [ ]:
# Creating the X and Y for training, the formula is set up to assume the target Y is the left most column = target_index=0
X, Y = create_X_Y(ts_scaled, lag=lag, n_ahead=n_ahead)

In [ ]:
# Spliting into train and test sets 
Xtrain, Ytrain = X[0:int(X.shape[0] * train_share)], Y[0:int(X.shape[0] * train_share)]
Xtest, Ytest = X[int(X.shape[0] * train_share):], Y[int(X.shape[0] * train_share):]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.layers import Dense,RepeatVector, LSTM, Dropout
from tensorflow.keras.layers import Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.layers import Bidirectional, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import plot_model

In [ ]:
from keras.callbacks import ModelCheckpoint, TensorBoard, Callback, EarlyStopping
early_stop = EarlyStopping(monitor = "loss", mode = "min", patience = 7)
model = Sequential()
model.add(Conv1D(filters=256, kernel_size=2, activation='relu', input_shape=(24,5)))
model.add(Conv1D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(RepeatVector(30))
model.add(LSTM(units=100, return_sequences=True, activation='relu'))
model.add(Dropout(0.2))
model.add(LSTM(units=100, return_sequences=True, activation='relu'))
model.add(Dropout(0.2))
model.add(LSTM(units=100, return_sequences=True, activation='relu'))
model.add(LSTM(units=100, return_sequences=True, activation='relu'))
model.add(Bidirectional(LSTM(128, activation='relu')))
model.add(Dense(100, activation='relu'))
model.add(Dense(24))
model.compile(loss='mse', optimizer='adam')

In [ ]:
history = model.fit(Xtrain,Ytrain,epochs=50, verbose=1, callbacks = [early_stop] )

plt.plot(history.history['loss'], label='Training loss')
plt.legend()

In [ ]:
model.save("./cnn-lstm-1.hdf5")

In [ ]:
#predict based on test data
yhat = model.predict(Xtest)

In [ ]:
date=df['date_ech']

In [ ]:
# Creating the predictions date range
dt=pd.DataFrame(date)
days = dt.values[-len(yhat):-len(yhat) + n_ahead]
days_df = pd.DataFrame(days)

In [ ]:
#prepare resulting series for inverse scaling transformation
#pay attention we will select only the first prediction we have made, therefore [0] used to select this window (we have generated multiple prediction sequences of 144 steps ahead, starting from each interval step in the test dataset)
pred_n_ahead = pd.DataFrame(yhat[0])
actual_n_ahead = pd.DataFrame(Ytest[0])

#repeat the column series 2 times, to make shape compatible for scale inversion
pr_p = pd.concat([pred_n_ahead]*5, axis=1)
ac_p = pd.concat([actual_n_ahead]*5, axis=1)

In [ ]:
#inverse scale tranform the series back to kiloWatts of power
pr_p = pd.DataFrame(scaler.inverse_transform(pr_p))
ac_p = pd.DataFrame(scaler.inverse_transform(ac_p))

#rename columns
pr_p = pr_p.rename(columns={0:'PredPower'})
ac_p = ac_p.rename(columns={0:'ActualPower'})

#concatenate together into one dataframe and set index
df_final = pd.concat([days_df, pr_p['PredPower'], ac_p['ActualPower']], axis=1).set_index(0)

In [ ]:
df_final[['PredPower','ActualPower']]

In [ ]:
#plot n_steps ahead for predicted and actual data
plt.figure(figsize=(15, 8))
plt.plot(df_final.index, df_final.ActualPower, color='C0', marker='o', label='Actual Power')
plt.plot(df_final.index, df_final.PredPower, color='C1', marker='o', label='Predicted Power', alpha=0.6)
plt.title('Predicted vs Actual Power')
plt.gcf().axes[0].yaxis.get_major_formatter().set_scientific(False)
plt.legend()
plt.savefig('forecast_example.png')
plt.show

In [ ]:
forecast_accuracy(df_final['PredPower'], df_final['ActualPower'], )